In [1]:
#importing required libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [44]:
# using beatiful soup to scrape Melbourne suburb data off Wikipedia
import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

In [45]:
#getting the right table
pc_table = soup.find('table', class_='wikitable sortable')
pc_table

<table class="wikitable sortable">
<tbody><tr>
<th>Suburb</th>
<th>Postcode</th>
<th>Local government area</th>
<th>Location<br/><sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2019)">citation needed</span></a></i>]</sup></th>
<th>Distance<sup class="reference" id="cite_ref-3"><a href="#cite_note-3">[3]</a></sup><br/><sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable sources. (May 2019)">citation needed</span></a></i>]</sup></th>
<th>Area<br/><sup class="noprint Inline-Template Template-Fact" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Citation_needed" title="Wikipedia:Citation needed"><span title="This claim needs references to reliable so

In [48]:
# Create dataframes
Suburb=[]
Postcode=[]
Gov=[]
Loc=[]
Dist=[]
Area=[]
Pop=[]
PopD=[]
Date=[]

# fill dataframes with data from table
for row in pc_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==9:
        Suburb.append(cells[0].find(text=True))
        Postcode.append(cells[1].find(text=True))
        Gov.append(cells[2].find(text=True))
        Loc.append(cells[3].find(text=True))
        Dist.append(cells[4].find(text=True))
        Area.append(cells[5].find(text=True))
        Pop.append(cells[6].find(text=True))
        PopD.append(cells[7].find(text=True))
        Date.append(cells[8].find(text=True))

In [49]:
# name the columns
df=pd.DataFrame(Suburb,columns=['Suburb'])
df['Postcode']=Postcode
df['Gov']=Gov
df['Loc']=Loc
df['Dist']=Dist
df['Area']=Area
df['Pop']=Pop
df['PopD']=PopD
df['Date']=Date
df

,Suburb,Postcode,Gov,Loc,Dist,Area,Pop,PopD,Date
0,Bellfield,3081,City of Banyule,None,None,0.9 km2,"1,793",None,
1,Briar Hill,3088,City of Banyule,None,None,None,"3,152",None,
2,Bundoora,3083,City of Banyule,None,None,15 km2,"28,653",None,
3,Eaglemont,3084,City of Banyule,None,None,1.9 km2,"3,873",None,
4,Eltham,3095,City of Banyule,None,None,None,None,None,
5,Eltham North,3095,City of Banyule,None,None,None,None,None,
6,Greensborough,3088,City of Banyule,None,None,None,None,None,
7,Heidelberg,3084,City of Banyule,None,None,None,None,None,
8,Heidelberg Heights,3081,City of Banyule,None,None,None,None,None,
9,Heidelberg West,3081,City of Banyule,None,None,None,None,None,


In [50]:
df=df.drop(["Gov","Loc","Dist","Area","Pop","PopD","Date"], axis=1)
df

,Suburb,Postcode
0,Bellfield,3081
1,Briar Hill,3088
2,Bundoora,3083
3,Eaglemont,3084
4,Eltham,3095
5,Eltham North,3095
6,Greensborough,3088
7,Heidelberg,3084
8,Heidelberg Heights,3081
9,Heidelberg West,3081


In [51]:
df=df[['Postcode', 'Suburb']]
df

,Postcode,Suburb
0,3081,Bellfield
1,3088,Briar Hill
2,3083,Bundoora
3,3084,Eaglemont
4,3095,Eltham
5,3095,Eltham North
6,3088,Greensborough
7,3084,Heidelberg
8,3081,Heidelberg Heights
9,3081,Heidelberg West


In [52]:
df = df.groupby(['Postcode'])['Suburb'].apply(", ".join).reset_index()
df = df.replace('\n','', regex=True)
df

,Postcode,Suburb
0,3000,Melbourne CBD
1,3002,East Melbourne
2,3003,West Melbourne
3,3004,Melbourne CBD
4,3006,"Southbank, South Wharf"
5,3008,Docklands
6,3011,"Footscray, Seddon"
7,3012,"Brooklyn, Kingsville, Maidstone, Tottenham, We..."
8,3013,Yarraville
9,3015,"Newport, Spotswood, South Kingsville"


In [83]:
df.dtypes

Postcode    object
Suburb      object
dtype: object

In [92]:
df[['Postcode']] = df[['Postcode']].apply(pd.to_numeric)
df[['Suburb']] = df[['Suburb']].astype(str)
df.dtypes

Postcode     int64
Suburb      object
dtype: object

In [89]:
# The code was removed by Watson Studio for sharing.

,postcode,lat,lon
0,200,-35.277272,149.117136
1,221,-35.201372,149.095065
2,800,-12.801028,130.955789
3,801,-12.801028,130.955789
4,804,-12.432181,130.843310


In [94]:
# get geo data
df_geo.rename(columns={'postcode':'Postcode', 'suburb':'Suburb', 'lat':'Lat', 'lon':'Lon'}, inplace=True)
df_geo.set_index("Postcode")
df.set_index("Postcode")
melbourne=pd.merge(df, df_geo)
melbourne.head()

,Postcode,Suburb,Lat,Lon
0,3000,Melbourne CBD,-37.814563,144.970267
1,3002,East Melbourne,-37.816640,144.987811
2,3003,West Melbourne,-37.806255,144.941123
3,3004,Melbourne CBD,-37.837324,144.976335
4,3006,"Southbank, South Wharf",-37.823258,144.965926


In [98]:
melbourne.drop_duplicates(subset='Postcode', keep='first', inplace=True)
melbourne

,Postcode,Suburb,Lat,Lon
0,3000,Melbourne CBD,-37.814563,144.970267
1,3002,East Melbourne,-37.816640,144.987811
2,3003,West Melbourne,-37.806255,144.941123
3,3004,Melbourne CBD,-37.837324,144.976335
4,3006,"Southbank, South Wharf",-37.823258,144.965926
5,3008,Docklands,-37.814719,144.948039
6,3011,"Footscray, Seddon",-37.799770,144.899587
9,3012,"Brooklyn, Kingsville, Maidstone, Tottenham, We...",-37.814624,144.847108
15,3013,Yarraville,-37.816178,144.889774
17,3015,"Newport, Spotswood, South Kingsville",-37.842477,144.883145


In [99]:
address = 'Melbourne, VIC, Australia'

geolocator = Nominatim(user_agent="melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [100]:
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(melbourne['Lat'], melbourne['Lon'], melbourne['Suburb']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

In [101]:
# define Foursquare credentials and version
CLIENT_ID = 'V3CSVKQZUKZAD1NYZEH10LDIWQOKGS4BZ3LCF3LBZMCTROJE' 
CLIENT_SECRET = '2HW52JE4O1XXGKYPDUTMNPOBI50RJYSC4PHVBOULBF20IX3Q' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V3CSVKQZUKZAD1NYZEH10LDIWQOKGS4BZ3LCF3LBZMCTROJE
CLIENT_SECRET:2HW52JE4O1XXGKYPDUTMNPOBI50RJYSC4PHVBOULBF20IX3Q


In [106]:
# create a function to explore neighborhoods through top venues in the area

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        limit=100
        radius=500
        url = 'https://api.foursquare.com/v2/venues/explore?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [107]:
# run the function
melbourne_venues = getNearbyVenues(names=melbourne['Suburb'],
                                   latitudes=melbourne['Lat'],
                                   longitudes=melbourne['Lon']
                                  )

Melbourne CBD
East Melbourne
West Melbourne
Melbourne CBD
Southbank, South Wharf
Docklands
Footscray, Seddon
Brooklyn, Kingsville, Maidstone, Tottenham, West Footscray
Yarraville
Newport, Spotswood, South Kingsville
Williamstown, Williamstown North
Altona, Seaholme
Braybrook
Albion, Sunshine, Sunshine North, Sunshine West
Albanvale, Kealba, Kings Park, St Albans
Ardeer
Cairnlea, Deer Park, Burnside, Burnside Heights, Caroline Springs, Ravenhall
Fieldstone, Mount Cottrell, Manor Lakes, Wyndham Vale, Mambourin
Altona North
Laverton North
Williams Landing
Altona Meadows, Laverton, Seabrook
Truganina, Hoppers Crossing, Tarneit
Derrimut, Point Cook, Werribee, Werribee South, Cocoroc
Flemington, Kensington
Maribyrnong, Ascot Vale, Travancore
Keilor East
Avondale Heights
Keilor, Keilor North
Calder Park, Delahey, Hillside, Sydenham, Taylors Hill
Keilor Downs, Keilor Lodge, Taylors Lakes
Moonee Ponds
Aberfeldie, Essendon, Essendon West
Essendon Fields, Essendon North, Strathmore, Strathmore He

In [108]:
# check resulting dataframe
print(melbourne_venues.shape)
melbourne_venues.head()

(2392, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Melbourne CBD,-37.814563,144.970267,Chin Chin,-37.815451,144.970189,Asian Restaurant
1,Melbourne CBD,-37.814563,144.970267,Pastuso,-37.815354,144.971062,Peruvian Restaurant
2,Melbourne CBD,-37.814563,144.970267,Coda,-37.815625,144.969906,Vietnamese Restaurant
3,Melbourne CBD,-37.814563,144.970267,Il Solito Posto,-37.814868,144.970290,Italian Restaurant
4,Melbourne CBD,-37.814563,144.970267,The French Brasserie,-37.815334,144.971370,French Restaurant


In [109]:
# see how many venues were returned for each neighborhood
melbourne_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbotsford,12,12,12,12,12,12
"Aberfeldie, Essendon, Essendon West",4,4,4,4,4,4
"Albert Park, Middle Park",24,24,24,24,24,24
"Albion, Sunshine, Sunshine North, Sunshine West",3,3,3,3,3,3
"Alphington, Fairfield",4,4,4,4,4,4
"Altona Meadows, Laverton, Seabrook",7,7,7,7,7,7
Altona North,3,3,3,3,3,3
"Altona, Seaholme",9,9,9,9,9,9
Armadale,11,11,11,11,11,11


In [110]:
# analyze each neighborhood

# one hot encoding
melbourne_onehot = pd.get_dummies(melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
melbourne_onehot['Neighborhood'] = melbourne_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [melbourne_onehot.columns[-1]] + list(melbourne_onehot.columns[:-1])
melbourne_onehot = melbourne_onehot[fixed_columns]

melbourne_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Cantonese Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Creperie,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hot Dog Joint,Hungarian Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Noodle House,Paella Restaurant,Pakistani Restaurant,Peruvian Restaurant,Pizza Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,South Indian Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Xinjiang Restaurant
0,Melbourne CBD,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Melbourne CBD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Melbourne CBD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Melbourne CBD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Melbourne CBD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [111]:
# group rows by suburb
melbourne_grouped = melbourne_onehot.groupby('Neighborhood').mean().reset_index()
melbourne_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Cantonese Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Creperie,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hot Dog Joint,Hungarian Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Noodle House,Paella Restaurant,Pakistani Restaurant,Peruvian Restaurant,Pizza Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,South Indian Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Xinjiang Restaurant
0,Abbotsford,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.0
1,"Aberfeldie, Essendon, Essendon West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.0
2,"Albert Park, Middle Park",0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.083333,0.00000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,

In [112]:
# function to sort most common restaurants in each neighborhood in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
# new dataframe with top 10 venues in each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
suburbs_restaurants_sorted = pd.DataFrame(columns=columns)
suburbs_restaurants_sorted['Suburb'] = melbourne_grouped['Neighborhood']

for ind in np.arange(melbourne_grouped.shape[0]):
    suburbs_restaurants_sorted.iloc[ind, 1:] = return_most_common_venues(melbourne_grouped.iloc[ind, :], num_top_venues)

suburbs_restaurants_sorted

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbotsford,Café,Vegetarian / Vegan Restaurant,Japanese Restaurant,Pizza Place,Sandwich Place,Greek Restaurant,Gastropub,Xinjiang Restaurant,Fish & Chips Shop,Dumpling Restaurant
1,"Aberfeldie, Essendon, Essendon West",Pizza Place,Thai Restaurant,Café,Bakery,Xinjiang Restaurant,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
2,"Albert Park, Middle Park",Café,Deli / Bodega,Italian Restaurant,Burger Joint,Restaurant,Middle Eastern Restaurant,Steakhouse,Bakery,Gastropub,Seafood Restaurant
3,"Albion, Sunshine, Sunshine North, Sunshine West",Deli / Bodega,Vietnamese Restaurant,Café,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
4,"Alphington, Fairfield",Thai Restaurant,Australian Restaurant,Fast Food Restaurant,Café,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
5,"Altona Meadows, Laverton, Seabrook",Fast Food Restaurant,Bakery,Italian Restaurant,Asian Restaurant,French Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop
6,Altona North,Deli / Bodega,Lebanese Restaurant,Mexican Restaurant,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
7,"Altona, Seaholme",Café,Italian Restaurant,Burger Joint,Pizza Place,Thai Restaurant,Seafood Restaurant,Snack Place,Fish & Chips Shop,Dumpling Restaurant,Eastern European Restaurant
8,Armadale,Café,Japanese Restaurant,Chinese Restaurant,Spanish Restaurant,Xinjiang Restaurant,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
9,"Ashburton, Ashwood",Café,Afghan Restaurant,Pizza Place,Fast Food Restaurant,Bakery,Fish & Chips Shop,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant


In [115]:
# clustering
kclusters = 7
melbourne_grouped_clustering = melbourne_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(melbourne_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 6, 0, 0, 0, 1, 4, 4, 3, 0], dtype=int32)

In [116]:
# change name of "Neighborhood" column to "Suburb" for merging
melbourne_grouped=melbourne_grouped.rename(columns = {'Neighborhood':'Suburb'})
melbourne_grouped

,Suburb,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Cantonese Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Creperie,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Halal Restaurant,Hot Dog Joint,Hungarian Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Noodle House,Paella Restaurant,Pakistani Restaurant,Peruvian Restaurant,Pizza Place,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,South Indian Restaurant,Souvlaki Shop,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Xinjiang Restaurant
0,Abbotsford,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.0
1,"Aberfeldie, Essendon, Essendon West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.0
2,"Albert Park, Middle Park",0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.083333,0.00000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.0000

In [118]:
# add clustering labels
suburbs_restaurants_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [124]:
# merge melbourne_grouped with melbourne to add latitude/longitude for each suburb
melbourne_merged=melbourne
melbourne_merged = melbourne_merged.join(suburbs_restaurants_sorted.set_index('Suburb'), on='Suburb')
melbourne_merged = melbourne_merged.dropna()
melbourne_merged.head()

,Postcode,Suburb,Lat,Lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3000,Melbourne CBD,-37.814563,144.970267,4.0,Café,Italian Restaurant,Asian Restaurant,Australian Restaurant,Sandwich Place,Japanese Restaurant,BBQ Joint,Sushi Restaurant,Dumpling Restaurant,Vietnamese Restaurant
1,3002,East Melbourne,-37.816640,144.987811,0.0,Café,Sushi Restaurant,Italian Restaurant,Breakfast Spot,Thai Restaurant,Restaurant,Japanese Restaurant,Steakhouse,Indian Restaurant,Australian Restaurant
2,3003,West Melbourne,-37.806255,144.941123,3.0,Café,Food Court,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop
3,3004,Melbourne CBD,-37.837324,144.976335,4.0,Café,Italian Restaurant,Asian Restaurant,Australian Restaurant,Sandwich Place,Japanese Restaurant,BBQ Joint,Sushi Restaurant,Dumpling Restaurant,Vietnamese Restaurant
4,3006,"Southbank, South Wharf",-37.823258,144.965926,4.0,Café,Australian Restaurant,Italian Restaurant,Restaurant,Greek Restaurant,Sandwich Place,Bakery,Sushi Restaurant,Steakhouse,Food Truck


In [125]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(melbourne_merged['Lat'], melbourne_merged['Lon'], melbourne_merged['Suburb'], melbourne_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [128]:
# examine the clusters
# cluster 1 - food heaven (everything from fine dining to food trucks)
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 0, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Melbourne,Café,Sushi Restaurant,Italian Restaurant,Breakfast Spot,Thai Restaurant,Restaurant,Japanese Restaurant,Steakhouse,Indian Restaurant,Australian Restaurant
5,Docklands,Café,Restaurant,Chinese Restaurant,Australian Restaurant,Middle Eastern Restaurant,Pizza Place,Sandwich Place,Sushi Restaurant,Japanese Restaurant,Thai Restaurant
15,Yarraville,Café,Pizza Place,Vietnamese Restaurant,Deli / Bodega,Bakery,Mexican Restaurant,Burger Joint,Fish & Chips Shop,Indian Restaurant,Chinese Restaurant
17,"Newport, Spotswood, South Kingsville",Café,Pizza Place,Thai Restaurant,Fast Food Restaurant,Bagel Shop,Xinjiang Restaurant,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
20,"Williamstown, Williamstown North",Café,Fish & Chips Shop,Italian Restaurant,Bakery,Restaurant,Pizza Place,Seafood Restaurant,Breakfast Spot,Fried Chicken Joint,Diner
27,"Albion, Sunshine, Sunshine North, Sunshine West",Deli / Bodega,Vietnamese Restaurant,Café,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
82,Moonee Ponds,Café,Vietnamese Restaurant,Italian Restaurant,Japanese Restaurant,Burger Joint,Deli / Bodega,Noodle House,Sandwich Place,Seafood Restaurant,Pizza Place
86,"Essendon Fields, Essendon North, Strathmore, S...",Café,Bakery,Pizza Place,Indian Restaurant,Sandwich Place,Chinese Restaurant,Fish & Chips Shop,Food,Restaurant,Gastropub
110,Parkville,Café,Japanese Restaurant,Middle Eastern Restaurant,Pizza Place,Indian Restaurant,Sandwich Place,Chinese Restaurant,Food Truck,Sushi Restaurant,Mexican Restaurant
124,"Coburg, Coburg North",Café,Kebab Restaurant,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Bakery,Xinjiang Restaurant,Food Court,Eastern European Restaurant,Ethiopian Restaurant


In [127]:
# examine the clusters
# cluster 2 - fast food in the suburbs
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 1, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,"Altona Meadows, Laverton, Seabrook",Fast Food Restaurant,Bakery,Italian Restaurant,Asian Restaurant,French Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop
71,Avondale Heights,Pizza Place,Asian Restaurant,Fast Food Restaurant,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop
193,Lower Plenty,Fast Food Restaurant,Pizza Place,Xinjiang Restaurant,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop
215,Templestowe,Fast Food Restaurant,Xinjiang Restaurant,Food Truck,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop,Food
226,Chirnside Park,Pizza Place,Fast Food Restaurant,Xinjiang Restaurant,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop
361,"Clarinda, Clayton South",Seafood Restaurant,Fast Food Restaurant,Xinjiang Restaurant,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop
367,"Springvale South, Dingley Village",Sandwich Place,Pizza Place,Fast Food Restaurant,Xinjiang Restaurant,Dim Sum Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
381,Rowville,Fast Food Restaurant,Steakhouse,Pizza Place,Dim Sum Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop
438,Frankston North,Pizza Place,Fast Food Restaurant,Bakery,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop


In [129]:
# examine the clusters
# cluster 3 - more fast food (pizza, asian, and dontus)
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 2, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
221,Donvale,Pizza Place,Xinjiang Restaurant,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop
379,"Doveton, Eumemmerring",Pizza Place,Xinjiang Restaurant,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop
710,Nyora,Pizza Place,Xinjiang Restaurant,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop


In [130]:
# examine the clusters
# cluster 4 - cafes, asian, and fish and chips
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 3, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,West Melbourne,Café,Food Court,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop
9,"Brooklyn, Kingsville, Maidstone, Tottenham, We...",Café,Food Truck,Xinjiang Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food
51,Williams Landing,Café,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food
123,Brunswick East,Café,Pizza Place,Xinjiang Restaurant,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop
192,Yarrambat,Café,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food
212,Balwyn North,Café,Thai Restaurant,Xinjiang Restaurant,Food Court,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
246,Burwood,Café,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food
303,Armadale,Café,Japanese Restaurant,Chinese Restaurant,Spanish Restaurant,Xinjiang Restaurant,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
358,Oakleigh South,Café,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food
440,Carrum Downs,Café,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food


In [131]:
# examine the clusters
# cluster 5 - inner city snacks
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 4, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Melbourne CBD,Café,Italian Restaurant,Asian Restaurant,Australian Restaurant,Sandwich Place,Japanese Restaurant,BBQ Joint,Sushi Restaurant,Dumpling Restaurant,Vietnamese Restaurant
3,Melbourne CBD,Café,Italian Restaurant,Asian Restaurant,Australian Restaurant,Sandwich Place,Japanese Restaurant,BBQ Joint,Sushi Restaurant,Dumpling Restaurant,Vietnamese Restaurant
4,"Southbank, South Wharf",Café,Australian Restaurant,Italian Restaurant,Restaurant,Greek Restaurant,Sandwich Place,Bakery,Sushi Restaurant,Steakhouse,Food Truck
6,"Footscray, Seddon",Vietnamese Restaurant,Café,Asian Restaurant,Bakery,Chinese Restaurant,African Restaurant,Restaurant,French Restaurant,Malay Restaurant,Falafel Restaurant
22,"Altona, Seaholme",Café,Italian Restaurant,Burger Joint,Pizza Place,Thai Restaurant,Seafood Restaurant,Snack Place,Fish & Chips Shop,Dumpling Restaurant,Eastern European Restaurant
24,Braybrook,Restaurant,Steakhouse,Italian Restaurant,Kebab Restaurant,Fast Food Restaurant,New American Restaurant,Pakistani Restaurant,Pizza Place,Indian Restaurant,Bakery
47,Altona North,Deli / Bodega,Lebanese Restaurant,Mexican Restaurant,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
55,"Truganina, Hoppers Crossing, Tarneit",Fast Food Restaurant,Café,Sandwich Place,Falafel Restaurant,Burger Joint,Xinjiang Restaurant,Diner,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
64,"Flemington, Kensington",Café,Bakery,Pizza Place,Malay Restaurant,Vietnamese Restaurant,Chinese Restaurant,Thai Restaurant,Ethiopian Restaurant,Noodle House,Sandwich Place
70,Keilor East,Italian Restaurant,Seafood Restaurant,Fish & Chips Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant


In [132]:
# examine the clusters
# cluster 6 - you're lucky to get anything but fish and chips
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 5, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
382,Scoresby,Fish & Chips Shop,Xinjiang Restaurant,Food Truck,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Food
441,Heatherton,Fish & Chips Shop,Xinjiang Restaurant,Food Truck,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Food


In [133]:
# examine the clusters
# cluster 7 - pizza, burgers, and dumplings
melbourne_merged.loc[melbourne_merged['Cluster Labels'] == 6, melbourne_merged.columns[[1] + list(range(5, melbourne_merged.shape[1]))]]

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,"Maribyrnong, Ascot Vale, Travancore",Pizza Place,Thai Restaurant,Indian Restaurant,Spanish Restaurant,Portuguese Restaurant,Greek Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
72,"Keilor, Keilor North",Pizza Place,Café,Bakery,Fish & Chips Shop,Xinjiang Restaurant,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
83,"Aberfeldie, Essendon, Essendon West",Pizza Place,Thai Restaurant,Café,Bakery,Xinjiang Restaurant,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
95,"Pascoe Vale, Pascoe Vale South",Bakery,Café,Pizza Place,Australian Restaurant,Xinjiang Restaurant,Food Truck,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
112,Carlton,Café,Pizza Place,Bakery,Breakfast Spot,Gastropub,Xinjiang Restaurant,Food Court,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
114,"Carlton North, Princes Hill",Café,Pizza Place,Bakery,Breakfast Spot,Gastropub,Xinjiang Restaurant,Food Court,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
146,"Clifton Hill, Fitzroy North",Café,Pizza Place,Bakery,Indian Restaurant,Fish & Chips Shop,Deli / Bodega,Greek Restaurant,Gastropub,Seafood Restaurant,Sushi Restaurant
162,Lalor,Pizza Place,Bakery,Xinjiang Restaurant,Food Truck,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop
173,Mill Park,Pizza Place,Café,Fish & Chips Shop,Burger Joint,Xinjiang Restaurant,Diner,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant
182,"Macleod, Yallambie",Pizza Place,Thai Restaurant,Café,Fish & Chips Shop,Burger Joint,Xinjiang Restaurant,Food Court,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
